tag data가 topic 모델링 하기에 적합한 형태인건가?
단문-갯수가 많은 데이터일때 부적합한점이라고 생각되는것은 없음...

항상 처음 겪는 어려움이 학습되어진 데이터를 가져오는데 그게 어떻게 이루어져서 나온건지
파악이 안됨... 한번 정리를 해놔야 함

day6 파일을 기준으로 파악해보자

In [ ]:
# 벡터라이징을 한 뒤 term frequency matrix 만드는 부분

In [5]:
corpus_fname = '../NPL/data/corpus_10days/news/2016-10-24_article_all_normed.txt'
dictionary_fname = './2016-10-24_article_all_normed_corpus.dictionary'
ldamodel_fname = './2016-10-24_article_all_normed_corpus_lda.pkl'
TRAIN_LDA = True

In [6]:
# 파일 열어 확인 
with open(corpus_fname, encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip()[:100], '... \n')

 ... 

의원 60명 내무장관에게 서한 철거 때 어린이 안전 신경 써야  보호자 없는 아동 난민 70명은 영국에 도착  22일 경찰에 돌 던지는 칼레 난민 연합뉴스  파리 런던 연합뉴스 박 ... 

경찰 로고 연합뉴스 캡처  해남 연합뉴스 박철홍 기자 23일 오후 10시 12분께 전남 해남군 해남읍의 박모 65 씨의 주택에서 불이 났다는 이웃 주민의 신고가 119 상황실에 접 ... 

상파울루 연합뉴스 김재순 통신원 브라질 경제의 침체 장기화에 따른 지방정부 재정난이 갈수록 확산하고 있다  23일 현지시간 브라질 도시협의회 에 따르면 연방정부를 통해 재정 현황을 ... 

로스앤젤레스 연합뉴스 장현구 특파원 미국 대통령 선거의 판세가 서서히 민주당 힐러리 클린턴 전 국무장관 쪽으로 기우는 상황에서 공화당 도널드 트럼프를 지지하는 메이저 신문이 처음으 ... 



In [7]:
# doublespaceLIneCorpus는 문서형식을 정해서 편리하게 사용하기 위한 class..
# 한줄이 하나의 doc이며 줄바꿈기호는 doublespace로 한다.. 
# iterator로 doc하나씩 출력함..
from soynlp.utils import DoublespaceLineCorpus
corpus_doc = DoublespaceLineCorpus(corpus_fname, iter_sent=False)

In [8]:
for i, sent in enumerate(corpus_doc):
    if i > 1 : break
    print(sent)

 

의원 60명 내무장관에게 서한 철거 때 어린이 안전 신경 써야  보호자 없는 아동 난민 70명은 영국에 도착  22일 경찰에 돌 던지는 칼레 난민 연합뉴스  파리 런던 연합뉴스 박성진 황정우 특파원 프랑스 칼레 난민촌 철거를 이틀 앞둔 22일 현지시간 난민촌 주변에서 현지 경찰과 난민이 충돌했다고 현지 프랑스앵포가 보도했다  난민촌 철거에 반대하는 난민 50명가량은 경찰을 향해 유리병과 돌을 던졌으며 경찰은 연막탄을 쏘면서 이들을 해산했다  이 과정에서 별다른 부상자는 나오지 않았다  프랑스 정부는 24일부터 약 일주일간에 걸쳐 불도저 등을 동원해 칼레 난민촌을 철거하고 이곳에 머무는 난민 6천여 명을 전국 난민 시설에 분산 수용할 계획이다  난민과 난민 지원 비정부기구가 철거 반대 운동을 벌일 수 있을 것으로 보고 프랑스 정부는 경찰을 증파해 현재 1천200여 명이 칼레에서 철거 작업에 대비하고 있다  또 칼레에서 가까운 벨기에도 난민들이 자국으로 넘어오지나 않을까 우려하며 자국과 프랑스 간 국경에 120명의 경찰을 배치했다  프랑스 칼레 난민촌의 모습 연합뉴스  프랑스 정부는 난민들에게 수천 장의 철거 안내서를 배부하고 난민촌 폐쇄 및 재배치 계획을 설명했다  영불 해협을 사이에 두고 영국과 마주 보는 프랑스 칼레에는 시리아 내전을 피하거나 중동 아프리카에서 건너온 난민이 몰려 살고 있다  이들은 상 하수 시설 화장실 등이 제대로 갖춰져 있지 않은 등 생활환경이 열악해 정글 로 불리는 난민촌에 머물며 상대적으로 일자리를 구하기 쉽고 영어를 사용하는 영국 밀입국을 시도하고 있다  한 프랑스 난민지원 단체는 2천 명가량의 난민이 여전히 영국행을 희망하며 난민촌에서 떠나길 거부하는 것으로 추산했다  프랑스 정부는 난민촌에서 떠나기를 거부하는 난민이 있다면 공권력을 행사할 수밖에 없다는 입장이다  세이브 더 칠드런 등 비정부기구와 영국 의원 60명은 난민촌 철거 시 어린이들이 안전할 수 있도록 프랑스 정부가 각별한 신경을 써 달라고 당부하는 서명이 담긴 서한

In [9]:
# 명사추출 dict 무슨 값이냐? 
# sklearn.countervectorizer tokenizer에 들어갈 값을 만들기위해.........
# custom_tokenize에서 하는일은?

import pickle
with open('../NPL/code/day6/2016-10-24-extract_noun_dict.pkl','rb') as f:
    noun_dict = pickle.load(f)

for i, dict in enumerate(noun_dict):
    if i > 10 : break
    print(dict, noun_dict[dict])    

되풀이 0.9926465053763441
마디 0.7234822207792208
대의 0.5425456249999999
위원 0.29266255555555554
장애 0.8842410404040405
패턴 0.772989748091603
중장기 0.9994146666666667
성사 0.9780263376623376
기후변화 0.9518623725490196
화장실 0.814938985915493
박하선 0.6119598235294118


In [10]:
# custom tokenizer 
# countvectorizer 인자로 들어가는 tokenizer를 만드는 이유?
# 명사만 추출해 주는것 같음.. 아무래도 tokenize니까... 
# 이 함수를 사용하는건 좀 더 알아보고 해야겠다.. 
# list comprehension 방식을 리딩할수 있어야한다...

def custom_tokenize(doc):
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in noun_dict:
                return subword
        return ''
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

In [11]:
# twitter 방식.. 이미 학습된 내용이며 평균이상의 결과물을 보이나 
# 도메인에따라 학습된 내용이 아닌 단어인 경우 결과가 좋지 않음.. 
# 최선의 방법은 도메인에 따라 학습된 것을 바탕으로 custom_tokenize를 돌려야 함..

from konlpy.tag import Twitter
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        pos = self.tagger.pos(sent)
#         pos = ['{}/{}'.format(word,tag) for word, tag in pos if tag=='Noun']
        pos = ['{}'.format(word) for word,tag in pos if tag=='Noun' and len(word)>1]
        return pos

tw_tokenizer = MyTokenizer(Twitter())
print(tw_tokenizer('테스트는 테스트로 테스트한다'))

['테스트', '테스트', '테스트']


In [12]:
# countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = tw_tokenizer)
x = vectorizer.fit_transform(corpus_doc)
idx2vocab = [vocab  for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [13]:
print(x.shape)
print(idx2vocab[0])

(26368, 52344)
가가


# gensim tutorial

LDA는 sklean과 gensim에 구현되어있는것 같은데 사용하는 data 형식이 다르다...
sparse / dense matrix에 대한 이해필요...

In [14]:
import scipy
print(scipy.__version__)
import gensim
print(gensim.__version__)

1.1.0
3.4.0


C:\Users\mediance_ssh\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\mediance_ssh\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [15]:
corpus_sparse = gensim.matutils.Sparse2Corpus(x, documents_columns=False)

for i, doc in enumerate(corpus_sparse):
    if i > 3: break
    print('#doc= %d: ' % i, doc[:10], '...' )

#doc= 0:  [] ...
#doc= 1:  [(9941, 1), (4602, 2), (3189, 1), (35025, 1), (1953, 3), (43868, 2), (9611, 1), (15603, 1), (34368, 1), (21675, 1)] ...
#doc= 2:  [(597, 1), (51304, 1), (9530, 1), (50513, 2), (1315, 1), (52039, 1), (51502, 1), (19214, 1), (1738, 1), (39423, 1)] ...
#doc= 3:  [(37890, 1), (21760, 1), (3965, 1), (30334, 1), (19333, 1), (35542, 1), (21062, 1), (20203, 1), (10601, 1), (8074, 1)] ...


In [16]:
print(len(corpus_sparse))

26368


In [17]:
# dictionary formats 야이.. 

In [18]:
from scipy.sparse import csr_matrix

row, col = x.nonzero()
data = [1]*len(row)
x_boolean = csr_matrix((data, (row, col)))
df = x_boolean.sum(axis=0)

n_doc = x.shape[0]
word2index = vectorizer.vocabulary_
df = df.tolist()[0]

In [19]:
# dictionary 작성...
with open(dictionary_fname, 'w', encoding='utf-8') as f:
    f.write('%d\n' % n_doc)
    for word, idx in word2index.items():
        f.write('%d\t%s\t%d\n' % (idx, word, df[idx]))

In [20]:
from gensim.corpora import Dictionary
dictionary = Dictionary.load_from_text(dictionary_fname)

In [21]:
print(dictionary.doc2bow(['코코모']))

[(44692, 1)]


In [22]:
%%time
from gensim.models import LdaModel

if TRAIN_LDA:
    ldamodel = LdaModel(corpus=corpus_sparse, num_topics=20, id2word=dictionary)
    import pickle
    with open(ldamodel_fname, 'wb') as f:
        pickle.dump(ldamodel, f)
else: 
    import pickle
    with open(ldamodel_fname, 'rb') as f:
        ldamodel = pickle.load(f)        

Wall time: 15.8 s


In [23]:
print(ldamodel.print_topic(10,topn=5))
print(ldamodel.get_topic_terms(1))

0.013*"학생" + 0.010*"교육" + 0.009*"클린턴" + 0.009*"트럼프" + 0.008*"무단"
[(17303, 0.021506276), (14481, 0.012684202), (20734, 0.010315268), (29811, 0.009976654), (5362, 0.008819833), (11198, 0.008548199), (8608, 0.007655176), (25877, 0.0067134677), (36604, 0.006641853), (11725, 0.005895578)]


In [24]:
def parse_topic_words(topic_str):
    return [col.split('*"')[1][:-1] for col in topic_str.split(' + ')]

In [25]:
for i in range(20):
    print('#topic= %d:' % i, parse_topic_words( ldamodel.print_topic(i, topn=5)))
    if i % 5 == 4: print()

#topic= 0: ['한국', '사업', '기업', '기술', '지원']
#topic= 1: ['방송', '모습', '사람', '연기', '기자']
#topic= 2: ['북한', '미국', '중국', '대화', '우리']
#topic= 3: ['뉴스', '무단', '전재', '금지', '제보']
#topic= 4: ['일보', '교수', '독도', '한국', '일본']

#topic= 5: ['스포츠서울', '예산', '경제', '정부', '내년']
#topic= 6: ['경찰', '기자', '무단', '금지', '배포']
#topic= 7: ['재단', '의혹', '대통령', '대표', '회의']
#topic= 8: ['영화', '네이버', '닥터', '개봉', '앵커']
#topic= 9: ['뉴스', '기자', '국회', '서울', '배포']

#topic= 10: ['학생', '교육', '클린턴', '트럼프', '무단']
#topic= 11: ['롯데', '리포트', '치료', '면세점', '그룹']
#topic= 12: ['남자', '기자', '감독', '소속사', '한국']
#topic= 13: ['억원', '삼성', '시장', '투자', '한남동']
#topic= 14: ['고객', '서비스', '만원', '금지', '뉴스']

#topic= 15: ['서울', '코리아', '기자', '경제', '오후']
#topic= 16: ['문화', '사람', '한국', '작품', '요리']
#topic= 17: ['개헌', '대통령', '국회', '논의', '국민']
#topic= 18: ['지역', '아파트', '단지', '가구', '시설']
#topic= 19: ['기자', '트와이스', '공개', '사진', '스포츠서울']



In [26]:
ldamodel.expElogbeta.shape

(20, 52344)

In [27]:
beta = ldamodel.expElogbeta.copy()

In [28]:
topn =20
important_terms = beta.argsort(axis=1)[:,-topn:]
print(important_terms.shape)

(20, 20)


In [29]:
for topic_idx in range(beta.shape[0]):
    if topic_idx % 5 == 0:
        print()
    term_indices = important_terms[topic_idx,:].reshape(-1)
    terms = reversed([idx2vocab[idx] for idx in term_indices])
    print('topic #{}: {}'.format(topic_idx, ' '.join(terms)))


topic #0: 한국 사업 기업 기술 지원 경제 세계 산업 지역 통해 아시아 협력 개발 위해 문화 연구 사회 이번 국내 분야
topic #1: 방송 모습 사람 연기 기자 드라마 뉴미디어 시청자 자신 라며 일간스포츠 에서 우리 사진 이야기 발매 스타 대한 장면 촬영
topic #2: 북한 미국 중국 대화 우리 정부 기자 이번 역도 배포 한국 대북 외교부 금지 장관 시립대 지난 행사 제재 전재
topic #3: 뉴스 무단 전재 금지 제보 배포 기자 사진 영상 서울 세종 어선 중국 의회 오전 오후 의장 주석 해경 충북도
topic #4: 일보 교수 독도 한국 일본 기증 삼성 지난 대구 총장 마약 미국 아들 기자 현지 대한 배포 금지 사실 필리핀

topic #5: 스포츠서울 예산 경제 정부 내년 우리 국민 임금 문화 위해 수술 임신 지원 편성 출산 한그루 지난 사회 문제 계획
topic #6: 경찰 기자 무단 금지 배포 뉴스 서울 병원 혐의 전재 도서관 조사 사건 대구 왕소 영장 혼술 오후 연인 지난
topic #7: 재단 의혹 대통령 대표 회의 최순실 입장 최씨 청와대 대해 기자 스포츠 대한 정부 문제 검찰 수사 사실 장관 미르
topic #8: 영화 네이버 닥터 개봉 앵커 나리 고양이 트레인 난민 기자 그린 관객 공개 대감 네팔 눈빛 마블 기술 코미디 럭키
topic #9: 뉴스 기자 국회 서울 배포 금지 전재 무단 대통령 의원 대표 연설 개헌 박근혜 시정 여의도 예산안 민주당 본회의 국민의당

topic #10: 학생 교육 클린턴 트럼프 무단 학교 후보 시간 취업 금지 교육청 대한 전재 경기 기자 장애인 지난 모모 체험 시험
topic #11: 롯데 리포트 치료 면세점 그룹 회장 기자 자전거 년생 무단 유로 일보 인체 호텔 치매 검사 전자신문 배포 수사 금지
topic #12: 남자 기자 감독 소속사 한국 무단 인천 아시아 코리아 인스타그램 배포 금지 테디 통해 전재 연인 김도훈 지난 제공 시사회
topic #13: 억원 삼성 시장 투자 한남동 기자 기준 

In [30]:
# pyLDAvis

In [31]:
import pyLDAvis
print(pyLDAvis.__version__)

2.1.2


In [42]:
import pyLDAvis.gensim as gensimvis
prepared_data = gensimvis.prepare(ldamodel, corpus_sparse, dictionary)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [43]:
pyLDAvis.display(prepared_data)

In [ ]:
# test end...

gensim의 topic modeling을 이용하기 위해서 전처리와 sparse matrix 변환을 해봄
전처리에서 custom tokenizer 사용은 품질관리에 큰 영향을 주는것 같음... postagging이 중요한거라면... 이것도 따로 적용한걸 써야함.. 
sparse matrix는 gensim에서 정해저있는 방식으로 값을 넘겨야해서.. 